Assignment 05 - Implement the Continuous Bag of Words (CBOW) Model

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [3]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)


In [4]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [5]:
test_sentence

['When',
 'forty',
 'winters',
 'shall',
 'besiege',
 'thy',
 'brow,',
 'And',
 'dig',
 'deep',
 'trenches',
 'in',
 'thy',
 "beauty's",
 'field,',
 'Thy',
 "youth's",
 'proud',
 'livery',
 'so',
 'gazed',
 'on',
 'now,',
 'Will',
 'be',
 'a',
 "totter'd",
 'weed',
 'of',
 'small',
 'worth',
 'held:',
 'Then',
 'being',
 'asked,',
 'where',
 'all',
 'thy',
 'beauty',
 'lies,',
 'Where',
 'all',
 'the',
 'treasure',
 'of',
 'thy',
 'lusty',
 'days;',
 'To',
 'say,',
 'within',
 'thine',
 'own',
 'deep',
 'sunken',
 'eyes,',
 'Were',
 'an',
 'all-eating',
 'shame,',
 'and',
 'thriftless',
 'praise.',
 'How',
 'much',
 'more',
 'praise',
 "deserv'd",
 'thy',
 "beauty's",
 'use,',
 'If',
 'thou',
 'couldst',
 'answer',
 "'This",
 'fair',
 'child',
 'of',
 'mine',
 'Shall',
 'sum',
 'my',
 'count,',
 'and',
 'make',
 'my',
 'old',
 "excuse,'",
 'Proving',
 'his',
 'beauty',
 'by',
 'succession',
 'thine!',
 'This',
 'were',
 'to',
 'be',
 'new',
 'made',
 'when',
 'thou',
 'art',
 'old,',
 

In [6]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}
vocab

{"'This",
 'And',
 'How',
 'If',
 'Proving',
 'Shall',
 'Then',
 'This',
 'Thy',
 'To',
 'Were',
 'When',
 'Where',
 'Will',
 'a',
 'all',
 'all-eating',
 'an',
 'and',
 'answer',
 'art',
 'asked,',
 'be',
 'beauty',
 "beauty's",
 'being',
 'besiege',
 'blood',
 'brow,',
 'by',
 'child',
 'cold.',
 'couldst',
 'count,',
 'days;',
 'deep',
 "deserv'd",
 'dig',
 "excuse,'",
 'eyes,',
 'fair',
 "feel'st",
 'field,',
 'forty',
 'gazed',
 'held:',
 'his',
 'in',
 'it',
 'lies,',
 'livery',
 'lusty',
 'made',
 'make',
 'mine',
 'more',
 'much',
 'my',
 'new',
 'now,',
 'of',
 'old',
 'old,',
 'on',
 'own',
 'praise',
 'praise.',
 'proud',
 'say,',
 'see',
 'shall',
 'shame,',
 'small',
 'so',
 'succession',
 'sum',
 'sunken',
 'the',
 'thine',
 'thine!',
 'thou',
 'thriftless',
 'thy',
 'to',
 "totter'd",
 'treasure',
 'trenches',
 'use,',
 'warm',
 'weed',
 'were',
 'when',
 'where',
 'winters',
 'within',
 'worth',
 "youth's"}

In [7]:
word_to_ix

{'so': 0,
 'own': 1,
 'mine': 2,
 'praise': 3,
 'lies,': 4,
 'small': 5,
 'where': 6,
 'thy': 7,
 "'This": 8,
 'besiege': 9,
 'now,': 10,
 'thou': 11,
 'and': 12,
 'sunken': 13,
 'eyes,': 14,
 'dig': 15,
 'sum': 16,
 'being': 17,
 'it': 18,
 'field,': 19,
 'when': 20,
 'blood': 21,
 'Were': 22,
 'fair': 23,
 'all': 24,
 'an': 25,
 "deserv'd": 26,
 'of': 27,
 "youth's": 28,
 'succession': 29,
 'old': 30,
 'Shall': 31,
 'thine!': 32,
 'worth': 33,
 'new': 34,
 'livery': 35,
 'This': 36,
 "excuse,'": 37,
 'much': 38,
 'Will': 39,
 'old,': 40,
 'his': 41,
 'asked,': 42,
 'forty': 43,
 "totter'd": 44,
 'within': 45,
 'count,': 46,
 'be': 47,
 'To': 48,
 'Where': 49,
 'use,': 50,
 'answer': 51,
 'praise.': 52,
 'Thy': 53,
 'And': 54,
 'Then': 55,
 'my': 56,
 'proud': 57,
 'gazed': 58,
 'If': 59,
 'held:': 60,
 'make': 61,
 'treasure': 62,
 'by': 63,
 'all-eating': 64,
 'in': 65,
 'to': 66,
 'thriftless': 67,
 'more': 68,
 'thine': 69,
 'a': 70,
 'warm': 71,
 'see': 72,
 'say,': 73,
 'deep': 

In [8]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

In [9]:
CONTEXT_SIZE = 2
EMDEDDING_DIM = 100

raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

In [10]:
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word:ix for ix, word in enumerate(vocab)}
ix_to_word = {ix:word for ix, word in enumerate(vocab)}

In [11]:
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
data[:1]

[(['We', 'are', 'to', 'study'], 'about')]

In [12]:
class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()
        
        #out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)
        

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.tensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)

In [13]:
model = CBOW(vocab_size, EMDEDDING_DIM)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [14]:
for epoch in range(50):
    total_loss = 0

    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)  

        log_probs = model(context_vector)

        total_loss += loss_function(log_probs, torch.tensor([word_to_ix[target]]))

    #optimize at the end of each epoch
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

In [15]:
#TESTING
context = ['People','create','to', 'direct']
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

In [16]:
#Print result
print(f'Raw text: {" ".join(raw_text)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')

Raw text: We are about to study the idea of a computational process. Computational processes are abstract beings that inhabit computers. As they evolve, processes manipulate other abstract things called data. The evolution of a process is directed by a pattern of rules called a program. People create programs to direct processes. In effect, we conjure the spirits of the computer with our spells.

Context: ['People', 'create', 'to', 'direct']

Prediction: programs
